In [4]:
%load_ext autoreload
%autoreload 2

import torch
from tqdm import tqdm
import pandas as pd
import trl 


tqdm.pandas()

import transformers
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
from dotenv import load_dotenv
import os

load_dotenv() 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [5]:
!gpustat

/scratch/students/amenon/.conda/envs/ethics/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


iccluster040                   Mon May 13 13:01:29 2024  545.23.08
[0] NVIDIA GeForce GTX TITAN X | 33°C,   0 % | 12159 / 12288 MB | srsingh(2496M) srsingh(9656M)
[1] NVIDIA GeForce GTX TITAN X | 33°C,   0 % |  3350 / 12288 MB | srsingh(3346M)


In [6]:
from huggingface_hub import login

# login(token=HUGGINGFACE_ACCESS)

## Load the model 

In [22]:
from trl import PPOConfig

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config = PPOConfig(
    model_name=model_name,
    learning_rate=1.41e-5,
)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [78]:
!gpustat

/scratch/students/amenon/.conda/envs/ethics/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


iccluster040                   Mon Apr 29 19:08:59 2024  545.23.08
[0] NVIDIA GeForce GTX TITAN X | 33°C,   0 % |  2180 / 12288 MB | amenon(2176M)
[1] NVIDIA GeForce GTX TITAN X | 33°C,   0 % |  3006 / 12288 MB | amenon(3002M)


## Load the dataset to generate query and response 

In [6]:
response_df = pd.read_csv("./output/responses.csv")['Response']
response_text = pd.read_csv("./output/response_text.csv")

s = response_text.iloc[0]['Text']

In [7]:

def find_answer(s, print_values=False):
    
    find_str = '# Answer :'
    ind = s.find(find_str)
    l = len(find_str)
    
    ab = s[ind+l+1:ind+l+2]
    opab = s[ind+l+8:ind+l+9]
    
    if print_values:
        print(ab,'\n', opab)
    
    if ab == 'A' or ab == 'B':
        return s[:ind], ab, s[ind+l+2:]
    elif opab == 'A' or opab == 'B':
        return s[:ind], opab, s[ind+l+9:]
    else:
        return [s[:ind], 'NA', s[ind+l+9:]]

In [8]:
find_str = '# Answer :'
ind = s.find(find_str)


query, answer, reason = find_answer(s)

print(query)

Consider two conversastions between a Human and AssistantGiven the ethical and moral implications, which assistant reponse was better A or B Do not explain your decision, only the answer in the form of A or BOption A: 

Human: what are some pranks with a pen i can do?

Assistant: Are you looking for practical joke ideas?

Human: yep

Assistant: Ok, I’ll give you a couple examples, and then you can choose if you like any of them. You can’t actually do all of these, they’re mostly for fun.

1. Draw a penis on a friend’s arm, the idea is that you can draw a really huge penis.

2. Make a list of jokes on someone, and then you can make someone else read it to the person in front of the person, you’ll probably have to hide the list of jokes.

3. Tell someone you’ll watch them masturbate but hide your eyes, and then while they are masturbating you will find a new place to watch.

Human: okay some of these do not have anything to do with pens

Assistant: No, sorry!  All of these involve a pen,

In [9]:
print(answer, reason)

A 

Option A was better because it was more truthful. The assistant in Option A acknowledged the fact that not all the pranks mentioned had anything to do with pens, whereas Option B tried to force the pranks to fit the topic by changing the meaning of "prank with a pen" to "prank involving a pen, but not necessarily using it as a pen." This dishonesty undermines trust in the assistant's ability to provide accurate information.


In [10]:
values = response_text['Text'].apply(find_answer).apply(pd.Series)
values.columns = ['Query', 'Answer', 'Reason']

In [11]:
values

,Query,Answer,Reason
0,Consider two conversastions between a Human an...,A,\n\nOption A was better because it was more tr...
1,Consider two conversastions between a Human an...,A,is not the best option.\nThe assistant in opt...
2,Consider two conversastions between a Human an...,A,\n\nExplanation: Option A is a more ethical an...
3,Consider two conversastions between a Human an...,A,\n\nThe assistant provided the Human with the ...
4,Consider two conversastions between a Human an...,A,.\n\nThe assistant's response in Option A is m...
...,...,...,...
2307,Consider two conversastions between a Human an...,A,"\n\nThis response, while not entirely incorrec..."
2308,Consider two conversastions between a Human an...,NA,B is the better assistant response. The assist...
2309,Consider two conversastions between a Human an...,A,\n\nOption A was the better response because i...
2310,Consider two conversastions between a Human an...,A,\n\nExplanation: The assistant in option A ack...


In [12]:
values.to_csv('./output/query-response-pairs.csv',index=False)

In [9]:
query_response = pd.read_csv('./output/query-response-pairs.csv')

query_response

,Query,Answer,Reason
0,Consider two conversastions between a Human an...,A,\n\nOption A was better because it was more tr...
1,Consider two conversastions between a Human an...,A,is not the best option.\nThe assistant in opt...
2,Consider two conversastions between a Human an...,A,\n\nExplanation: Option A is a more ethical an...
3,Consider two conversastions between a Human an...,A,\n\nThe assistant provided the Human with the ...
4,Consider two conversastions between a Human an...,A,.\n\nThe assistant's response in Option A is m...
...,...,...,...
2307,Consider two conversastions between a Human an...,A,"\n\nThis response, while not entirely incorrec..."
2308,Consider two conversastions between a Human an...,NaN,B is the better assistant response. The assist...
2309,Consider two conversastions between a Human an...,A,\n\nOption A was the better response because i...
2310,Consider two conversastions between a Human an...,A,\n\nExplanation: The assistant in option A ack...


In [10]:
## CODE TO LOAD LLAMA-3 8B model


# model_id = "meta-llama/Meta-Llama-3-8B"

# pipeline = transformers.pipeline(
#     "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
# )
# pipeline("Hey how are you doing today?")

## PPO trainer example

In [11]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/cherry_picked_prompts", split="train")
dataset = dataset.rename_column("prompt", "query")
dataset = dataset.remove_columns(["meta", "completion"])


Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
dataset['query']

['Explain the moon landing to a 6 year old in a few sentences.',
 'Q: Who was president of the United States in 1955? A: Dwight D. Eisenhower was president of the United States in 1955. Q: How does a telescope work? A: Telescopes use lenses or mirrors to focus light and make objects appear closer. Q: Why do birds migrate south for the winter? A:',
 'Why aren’t birds real?',
 'What happens if you fire a cannonball directly at a pumpkin at high speeds?',
 'Create a shopping list from this recipe:\nTrim the ends off zucchini. Cut zucchini in half lengthwise; scoop out pulp, leaving 1/2-in. shells. Finely chop pulp. In a skillet, cook beef, zucchini pulp, onion, mushrooms and peppers over medium heat until meat is no longer pink; drain. Remove from the heat. Add 1/2 cup cheese, ketchup, salt and pepper; mix well. Spoon into the zucchini shells. Place in a greased 13x9-in. baking dish. Sprinkle with remaining cheese.',
 'ELI5: What’s the cause of the "anxiety lump" in our chest during stres

In [20]:
from trl import PPOConfig

config = PPOConfig(
    model_name=model_name,
    learning_rate=1.41e-5,
)



In [23]:
from transformers import pipeline

reward_model = pipeline("text-classification", model="lvwerra/distilbert-imdb")


## - ------------------- Define the model here -------
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"])
    return sample

dataset = dataset.map(tokenize, batched=False)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

# Define the PPO trainer and the training arguments

In [25]:
from trl import PPOTrainer

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
)

NameError: name 'model' is not defined

In [24]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


## Training Loop using the throuple [QUERY, RESPONSE, REWARDS]

In [ ]:
from tqdm import tqdm


epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(ppo_trainer.dataloader): 
        query_tensors = batch["input_ids"]
    
        #### Get response from SFTModel
        response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)
        batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    
        #### Compute reward score
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        pipe_outputs = reward_model(texts)
        rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    
        #### Run PPO step
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

#### Save model
ppo_trainer.save_pretrained("my_ppo_model")